In [7]:
import pandas as pd

In [9]:
# 1. Cargar el dataset
df = pd.read_csv('../output/sales_data_clean.csv', parse_dates=['orderdate'])

In [10]:
# 2. Preparar columnas temporales
df['Quarter'] = df['orderdate'].dt.to_period('Q')
df['Year'] = df['orderdate'].dt.year
df['Month'] = df['orderdate'].dt.month

In [11]:
# 3. Ventas totales por trimestre
sales_by_quarter = (
    df.groupby('Quarter')['sales']
    .sum()
    .reset_index()
    .rename(columns={'sales': 'TotalSales'})
)
sales_by_quarter['Growth'] = sales_by_quarter['TotalSales'].pct_change()

In [12]:
# 4. Ventas por línea de producto por trimestre
product_sales = (
    df.groupby(['Quarter', 'productline'])['sales']
    .sum()
    .reset_index()
    .rename(columns={'sales': 'ProductSales'})
)

In [13]:
# 5. Ventas por territorio y año
region_sales = (
    df.groupby(['territory', 'Year'])['sales']
    .sum()
    .unstack()
    .fillna(0)
)
region_sales['Growth_%'] = region_sales.pct_change(axis=1)[region_sales.columns[-1]] * 100
region_sales = region_sales.reset_index()

In [14]:
# 6. Ticket promedio por trimestre
df['Ticket'] = df['sales'] / df['quantityordered']
avg_ticket = (
    df.groupby('Quarter')['Ticket']
    .mean()
    .reset_index()
    .rename(columns={'Ticket': 'AvgTicket'})
)

In [15]:
# 7. Detectar trimestre con mayor crecimiento
peak_quarter = sales_by_quarter.sort_values('Growth', ascending=False).iloc[0]

In [16]:
# 8. Línea de producto destacada en ese trimestre
top_product = (
    product_sales[product_sales['Quarter'] == peak_quarter['Quarter']]
    .sort_values('ProductSales', ascending=False)
    .iloc[0]
)

In [17]:
# 9. Región con mayor crecimiento interanual
top_region = region_sales.sort_values('Growth_%', ascending=False).iloc[0]

In [19]:
# 10. Construir resumen narrativo
print("📘 Insight Resumen\n")
print(f"Durante el {peak_quarter[ 'Quarter']}, las ventas totales alcanzaron los ${peak_quarter['TotalSales']:,.0f},")
print(f"lo que representa un crecimiento del {peak_quarter['Growth']:.1%} respecto al trimestre anterior.")
print(f"La categoría destacada fue *{top_product['productline']}*, con un total de ventas de ${top_product['ProductSales']:,.0f}.")
print(f"La región de {top_region['territory']} lideró el crecimiento interanual con un +{top_region['Growth_%']:.1f} %.")
print("Este comportamiento sugiere una respuesta positiva a las estrategias comerciales aplicadas durante el período.")

📘 Insight Resumen

Durante el 2003Q4, las ventas totales alcanzaron los $1,860,005,
lo que representa un crecimiento del 186.4% respecto al trimestre anterior.
La categoría destacada fue *Classic Cars*, con un total de ventas de $829,663.
La región de APAC lideró el crecimiento interanual con un +-57.2 %.
Este comportamiento sugiere una respuesta positiva a las estrategias comerciales aplicadas durante el período.
